# Multi-Stock Universal LSTM Training Notebook
This notebook downloads multiple stocks and trains a universal LSTM model.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

tickers=['AAPL','TSLA','NVDA','RELIANCE.NS','^NSEI','^NSEBANK','BTC-USD']
start='2015-01-01'
end=datetime.today().strftime('%Y-%m-%d')
raw_data={}
for t in tickers:
    df=yf.download(t,start=start,end=end)
    if not df.empty:
        raw_data[t]=df[['Close']].dropna()
raw_data

/tmp/ipython-input-2683179385.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download(t,start=start,end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2683179385.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download(t,start=start,end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2683179385.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download(t,start=start,end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2683179385.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download(t,start=start,end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2683179385.py:12: FutureWarning: YF.download() has changed argument auto_adjust defau

{'AAPL': Price            Close
 Ticker            AAPL
 Date                  
 2015-01-02   24.237549
 2015-01-05   23.554739
 2015-01-06   23.556961
 2015-01-07   23.887276
 2015-01-08   24.805077
 ...                ...
 2025-11-17  267.459991
 2025-11-18  267.440002
 2025-11-19  268.559998
 2025-11-20  266.250000
 2025-11-21  271.489990
 
 [2740 rows x 1 columns],
 'TSLA': Price            Close
 Ticker            TSLA
 Date                  
 2015-01-02   14.620667
 2015-01-05   14.006000
 2015-01-06   14.085333
 2015-01-07   14.063333
 2015-01-08   14.041333
 ...                ...
 2025-11-17  408.920013
 2025-11-18  401.250000
 2025-11-19  403.989990
 2025-11-20  395.230011
 2025-11-21  391.089996
 
 [2740 rows x 1 columns],
 'NVDA': Price            Close
 Ticker            NVDA
 Date                  
 2015-01-02    0.483038
 2015-01-05    0.474880
 2015-01-06    0.460482
 2015-01-07    0.459282
 2015-01-08    0.476560
 ...                ...
 2025-11-17  186.600006
 2025-11

In [ ]:
from sklearn.preprocessing import MinMaxScaler
SEQ_LEN=100
X_list=[]; y_list=[]
scalers={}
for t,df in raw_data.items():
    vals=df['Close'].values.reshape(-1,1)
    sc=MinMaxScaler((0,1))
    scaled=sc.fit_transform(vals)
    scalers[t]=sc
    for i in range(SEQ_LEN,len(scaled)):
        X_list.append(scaled[i-SEQ_LEN:i])
        y_list.append(scaled[i])
X=np.array(X_list); y=np.array(y_list)
X.shape, y.shape

((19274, 100, 1), (19274, 1))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.15,shuffle=True,random_state=42)
X_train.shape, X_val.shape

((16382, 100, 1), (2892, 100, 1))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout

model=Sequential([
 LSTM(128,input_shape=(SEQ_LEN,1)),
 Dropout(0.2),
 Dense(32,activation='relu'),
 Dense(1)
])
model.compile(optimizer='adam',loss='mse',metrics=['mae'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,721 (276.25 KB)

 Trainable params: 70,721 (276.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS=10
BATCH=128
history=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=EPOCHS,batch_size=BATCH)
history.history

Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0252 - mae: 0.0831 - val_loss: 3.8252e-04 - val_mae: 0.0122
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.0637e-04 - mae: 0.0189 - val_loss: 4.1339e-04 - val_mae: 0.0127
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 7.7721e-04 - mae: 0.0175 - val_loss: 4.7033e-04 - val_mae: 0.0141
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.7411e-04 - mae: 0.0163 - val_loss: 4.0579e-04 - val_mae: 0.0130
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 5.9957e-04 - mae: 0.0155 - val_loss: 2.8320e-04 - val_mae: 0.0105
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.4953e-04 - mae: 0.0148 - val_loss: 2.7188e-04 - val_mae: 0.0103
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 5.2249e-04 - mae: 0.0147 - val_loss: 3.6701e-04 - val_mae: 0.0126
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 4.9485e-04 - mae: 0.0143 - val_loss: 5.7986e-04 - val_mae: 

{'loss': [0.007432001177221537,
  0.0008921980625018477,
  0.0007579062366858125,
  0.0006527561345137656,
  0.0005912040360271931,
  0.0005328335100784898,
  0.0005190002266317606,
  0.00046590136480517685,
  0.0004573947808239609,
  0.00041442009387537837],
 'mae': [0.04132991284132004,
  0.018708843737840652,
  0.0172529648989439,
  0.01605229265987873,
  0.015315655618906021,
  0.014561858028173447,
  0.014608022756874561,
  0.013886537402868271,
  0.013820613734424114,
  0.013129327446222305],
 'val_loss': [0.00038251973455771804,
  0.0004133910988457501,
  0.00047033122973516583,
  0.0004057929618284106,
  0.0002831971796695143,
  0.0002718791365623474,
  0.00036700686905533075,
  0.0005798604688607156,
  0.0003366891760379076,
  0.0002227790537290275],
 'val_mae': [0.01217703614383936,
  0.012714648619294167,
  0.014100107364356518,
  0.013033955357968807,
  0.010456766933202744,
  0.01033723633736372,
  0.012612540274858475,
  0.016122058033943176,
  0.011972380802035332,
  0.0

In [ ]:
model.save('keras_model_universal.h5')
'File saved as keras_model_universal.h5'

'File saved as keras_model_universal.h5'